## 05 RESNET

In [34]:
import torch
from PIL import Image
import pickle
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader
import time
import copy

In [21]:
with open('notchers_5x100', 'rb') as f:
    image_paths = pickle.load(f)

In [19]:
model = models.resnet18(pretrained = True)

#model preformulation - we have 5 classes
model.fc = nn.Linear(512,5)

#CPU to GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

#loss
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

#decay lr by 0.1 for every 10 epochs
schedular = lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)

In [20]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [59]:
data = torch.zeros((500, 3, 224, 224))
categories = 5
samples_per_category = 100
labels = []

for i in range(samples_per_category):
    for j in range(categories):
        path = image_paths[j][i]
        input_image = Image.open(path)
        inx = i*categories+j
        if inx % 10 == 0 and j == 0:
            print(f'{inx}/{categories*samples_per_category}')
        try:
            input_tensor = preprocess(input_image)
            data[inx] = input_tensor
        except:
            print(inx, path)
            print('Mistake')
        labels.append(j)
        


0/500
10/500
20/500
30/500
40/500
50/500
60/500
70/500
80/500
90/500
100/500
110/500
120/500
130/500
140/500
150/500
160/500
170/500
180/500
190/500
200/500
210/500
220/500
230/500
240/500
250/500
260/500
270/500
280/500
290/500
300/500
310/500
320/500
330/500
340/500
350/500
360/500
370/500
380/500
390/500
400/500
410/500
420/500
430/500
440/500
450/500
460/500
470/500
480/500
490/500


In [76]:
labels = np.array(labels)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2)

In [78]:
class Data(Dataset):
    
    def __init__(self, data, labels):
        self.x = data
        self.y = torch.from_numpy(labels)
        self.samples = data.shape[0]
        self.features = data.shape[1]
        
    def __getitem__(self, index):
        # define indexing
        return self.x[index], self.y[index]
        
    def __len__(self):
        # len(dataset)
        return self.samples

In [79]:
train = Data(X_train, y_train)
test = Data(X_test, y_test)
print(f'train: {len(train)}, test: {len(test)}')
train.y.size()

train: 400, test: 100


torch.Size([400])

In [80]:
datasets = {'train': train, 'test': test}
dataloaders = {x: DataLoader(datasets[x], batch_size=5, num_workers=0) for x in ['train', 'test']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'test']}
class_names = np.unique(labels)

In [81]:
class_names

array([0., 1., 2., 3., 4.], dtype=float32)

In [82]:
def train_model(model, criterion, optimizer, scheduler, num_epochs = 25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model


In [83]:
model = train_model(model, criterion, optimizer, schedular, num_epochs = 25)

Epoch 1/25
----------
train Loss: 1.7000 Acc: 0.2650
test Loss: 1.7530 Acc: 0.4400

Epoch 2/25
----------
train Loss: 1.2513 Acc: 0.4825
test Loss: 1.6125 Acc: 0.4600

Epoch 3/25
----------
train Loss: 0.8916 Acc: 0.6675
test Loss: 1.8081 Acc: 0.4100

Epoch 4/25
----------
train Loss: 0.5115 Acc: 0.8375
test Loss: 2.2694 Acc: 0.3900

Epoch 5/25
----------
train Loss: 0.3851 Acc: 0.8800
test Loss: 2.5184 Acc: 0.3700

Epoch 6/25
----------
train Loss: 0.2638 Acc: 0.9225
test Loss: 2.3425 Acc: 0.3700

Epoch 7/25
----------
train Loss: 0.1979 Acc: 0.9350
test Loss: 2.2481 Acc: 0.3800

Epoch 8/25
----------
train Loss: 0.1321 Acc: 0.9525
test Loss: 2.3980 Acc: 0.4300

Epoch 9/25
----------
train Loss: 0.0907 Acc: 0.9775
test Loss: 2.4336 Acc: 0.3700

Epoch 10/25
----------
train Loss: 0.0760 Acc: 0.9800
test Loss: 2.5275 Acc: 0.3800

Epoch 11/25
----------
train Loss: 0.0386 Acc: 0.9925
test Loss: 2.5804 Acc: 0.3600

Epoch 12/25
----------
train Loss: 0.0236 Acc: 0.9925
test Loss: 2.6041 Ac